---
#### 작업형 1-1. 종량제 봉투종류가 규격봉토이고, 용도가 음식물쓰레기인 2L 가격평균을 구하라. (가격=0 제외, 반올림하여 정수 출력)
---

In [5]:
import numpy as np
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch5/data5-1.csv")
print(df.shape)
df.head(2)

df_new = df[
    (df['종량제봉투종류'] == '규격봉투') & (df['종량제봉투용도'] == '음식물쓰레기') & (df['2ℓ가격'] != 0)
]
print('정답 = ', round(df_new['2ℓ가격'].mean()) )

(762, 26)
정답 =  118


---
#### 작업형 1-2. bmi 수치가 정상인 사람수와 위험체중인 사람수의 차이를 절대값으록 구하라 (정수출력)
- bmi = weight / (height/100)**2
- 정상 = 18.5이상 ~ 23미만, 위험 = 23이상 ~ 25미만
---

In [18]:
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch5/data5-2.csv")
print(df.shape)
df.head(2)

df['bmi'] = df['Weight'] / (df['Height']/100)**2

normal = df[
    (df['bmi'] >= 18.5) & (df['bmi'] < 23)
]
danger = df[
    (df['bmi'] >= 23) & (df['bmi'] < 25)
]

print('정답 = ', int(abs(len(normal) - len(danger))))

(10000, 3)
정답 =  144


---
#### 작업형 1-3. 순 전입학생이 가장 많은 학교의 전체 학생수를 구하라
- 순 전입학생 = 전입학생 - 전출학생
---

In [29]:
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch5/data5-3.csv")
print(df.shape)

df['순전입(계)'] = df['전입학생수(계)'] - df['전출학생수(계)']
tmp = df.sort_values('순전입(계)', ascending=False)
print('정답 = ', int(tmp.iloc[0,-2]))

(6309, 30)
정답 =  230


---
#### 작업형 2-1. 차량의 특징기반 데이터를 활용해 test 데이터의 중고차 가격을 예측하라
- 타겟: price
- 평가: RMSE
- 제출: result.csv (pred)
---

In [42]:
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch5/train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch5/test.csv")
print(train.shape, test.shape)
display(train.head(2), test.head(2))

############################ EDA ##########################################################
# 숫자형, 범주형 변수 혼재. 결측치 없음
# train/test obj변수 카테고리 동일
###########################################################################################

# 1.타겟분리
y = train['price']
train = train.drop('price', axis=1)

# 2.수치형변수 스케일링
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
scaler = StandardScaler()
num_cols = ['mileage', 'tax', 'mpg', 'engineSize']
train[num_cols] = scaler.fit_transform(train[num_cols])
test[num_cols] = scaler.transform(test[num_cols])

# 3.범주형변수 원-핫 인코딩
train = pd.get_dummies(train)
test = pd.get_dummies(test)

# 4.검증데이터 분리
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train, y, test_size=0.2, random_state=42)

# 5. model,eval
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error
model = RandomForestRegressor(random_state=120)
model.fit(X_train, y_train)

model_pred = model.predict(X_valid)
score = root_mean_squared_error(y_valid, model_pred)
print('RMSE = ', score)

(3759, 9) (1617, 8)


,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,EcoSport,2017,10298,Manual,25013,Petrol,150,53.3,1.0
1,Focus,2016,10491,Manual,30970,Diesel,0,74.3,1.5


,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,S-MAX,2016,Manual,23905,Diesel,125,56.5,2.0
1,Fiesta,2018,Manual,16895,Petrol,145,40.3,1.5


RMSE =  1431.282924867332


In [43]:
## LightGBM
from lightgbm import LGBMRegressor
model2 = LGBMRegressor(random_state=120)
model2.fit(X_train, y_train)

model2_pred = model2.predict(X_valid)
score = root_mean_squared_error(y_valid, model2_pred)
print('RMSE = ', score)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000774 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 398
[LightGBM] [Info] Number of data points in the train set: 3007, number of used features: 23
[LightGBM] [Info] Start training from score 12318.722980
RMSE =  1345.1260618964652


In [53]:
## XGBoost
from xgboost import XGBRegressor
model3 = XGBRegressor(random_state=120)
model3.fit(X_train, y_train)

model3_pred = model3.predict(X_valid)
score = root_mean_squared_error(y_valid, model3_pred)
print('RMSE = ', score)

RMSE =  1329.8104248046875


In [47]:
# submit
pred = model3.predict(test)
result = pd.DataFrame({
    'pred':pred
})
result.to_csv('result.csv', index=False)

In [50]:
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch5/train.csv")
train.describe(include='all')

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
count,3759,3759.000000,3759.000000,3759,3759.000000,3759,3759.000000,3759.000000,3759.000000
unique,19,NaN,NaN,3,NaN,3,NaN,NaN,NaN
top,Fiesta,NaN,NaN,Manual,NaN,Petrol,NaN,NaN,NaN
freq,1372,NaN,NaN,3234,NaN,2541,NaN,NaN,NaN
mean,NaN,2016.887736,12353.321362,NaN,23191.786379,NaN,114.444001,57.793083,1.352807
std,NaN,2.151811,4750.338051,NaN,19223.258713,NaN,61.713682,10.099221,0.437130
min,NaN,2002.000000,1095.000000,NaN,5.000000,NaN,0.000000,20.900000,0.000000
25%,NaN,2016.000000,9000.000000,NaN,9997.500000,NaN,75.000000,52.300000,1.000000
50%,NaN,2017.000000,11495.000000,NaN,18180.000000,NaN,145.000000,58.900000,1.200000
75%,NaN,2018.000000,15310.000000,NaN,31000.000000,NaN,145.000000,65.700000,1.500000
